# Class-Conditional Synthesis with Latent Diffusion Models

Download the checkpoint (~1.7 GB). This will usually take 1-2 minutes.

In [4]:
# make sure you are in root folder
%cd ..

C:\Users\DaPC\PycharmProjects\ldm1


In [ ]:
!mkdir -p models/ldm/cin256-v2/
!wget -O models/ldm/cin256-v2/model.ckpt https://ommer-lab.com/files/latent-diffusion/nitro/cin/model.ckpt

Load the model

In [2]:
#@title loading utils
import torch
from omegaconf import OmegaConf

from ldm.util import instantiate_from_config


def load_model_from_config(config, ckpt):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt)#, map_location="cpu")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    model.cuda()
    model.eval()
    return model


def get_model():
    config = OmegaConf.load("configs/latent-diffusion/cin256-v2.yaml")  
    model = load_model_from_config(config, "models/ldm/cin256-v2/model.ckpt")
    return model

In [5]:
from ldm.models.diffusion.ddim import DDIMSampler
model = get_model()
sampler = DDIMSampler(model)

Loading model from models/ldm/cin256-v2/model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 400.92 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels


And go. Quality, sampling speed and diversity are best controlled via the `scale`, `ddim_steps` and `ddim_eta` variables. As a rule of thumb, higher values of `scale` produce better samples at the cost of a reduced output diversity. Furthermore, increasing `ddim_steps` generally also gives higher quality samples, but returns are diminishing for values > 250. Fast sampling (i e. low values of `ddim_steps`) while retaining good quality can be achieved by using `ddim_eta = 0.0`.

In [4]:
import numpy as np 
from PIL import Image
from einops import rearrange
from torchvision.utils import make_grid
import os


classes = [25, 187, 448, 992]   # define classes to be sampled here
# classes = list(range(1000))  # this samples all classes in ImageNet
n_samples_per_class = 5

ddim_steps = 20
ddim_eta = 0.0
scale = 3.0   # for unconditional guidance


all_samples = list()

with torch.no_grad():
    with model.ema_scope():
        uc = model.get_learned_conditioning(
            {model.cond_stage_key: torch.tensor(n_samples_per_class*[1000]).to(model.device)}
            )
        
        for class_label in classes:
            print(f"rendering {n_samples_per_class} examples of class '{class_label}' in {ddim_steps} steps and using s={scale:.2f}.")
            xc = torch.tensor(n_samples_per_class*[class_label])
            c = model.get_learned_conditioning({model.cond_stage_key: xc.to(model.device)})
            
            samples_ddim, _ = sampler.sample(S=ddim_steps,
                                             conditioning=c,
                                             batch_size=n_samples_per_class,
                                             shape=[3, 64, 64],
                                             verbose=False,
                                             unconditional_guidance_scale=scale,
                                             unconditional_conditioning=uc, 
                                             eta=ddim_eta)

            x_samples_ddim = model.decode_first_stage(samples_ddim)
            x_samples_ddim = torch.clamp((x_samples_ddim+1.0)/2.0, 
                                         min=0.0, max=1.0)
            all_samples.append(x_samples_ddim)


rendering 5 examples of class '0' in 20 steps and using s=3.00.
Data shape for DDIM sampling is (5, 3, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|██████████| 20/20 [00:08<00:00,  2.27it/s]


rendering 5 examples of class '1' in 20 steps and using s=3.00.
Data shape for DDIM sampling is (5, 3, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|██████████| 20/20 [00:04<00:00,  4.13it/s]


rendering 5 examples of class '2' in 20 steps and using s=3.00.
Data shape for DDIM sampling is (5, 3, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


...

rendering 5 examples of class '896' in 20 steps and using s=3.00.
Data shape for DDIM sampling is (5, 3, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|██████████| 20/20 [00:05<00:00,  3.81it/s]


rendering 5 examples of class '897' in 20 steps and using s=3.00.
Data shape for DDIM sampling is (5, 3, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|██████████| 20/20 [00:05<00:00,  3.79it/s]


RuntimeError: CUDA out of memory. Tried to allocate 640.00 MiB (GPU 0; 8.00 GiB total capacity; 6.26 GiB already allocated; 0 bytes free; 6.93 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
# simple script to save the images locally
counter = 0
for x in all_samples:
    for z in x:
        imgpath = os.path.join("log_dir/models/ldm/cin256", f"sample_{counter:06}.png") #
        img = 255. * z.cpu().numpy()
        img = rearrange(img, 'c h w -> h w c')
        img = img.astype(np.uint8)
        img = Image.fromarray(img)
        img.save(imgpath)
        counter += 1

4485


In [ ]:
# display as grid

grid = torch.stack(all_samples, 0)
grid = rearrange(grid, 'n b c h w -> (n b) c h w')
grid = make_grid(grid, nrow=n_samples_per_class)

# to image
grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
Image.fromarray(grid.astype(np.uint8))